# Infeasibility UQ

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the QoI as the value of constraints, i.e. the infeasibility.

Dask is used to parallelise the evaluations for a SLURM cluster.


In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import shutil
import os

## Campaign to capture feasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals.

To start with, make just 4 inputs uncertain.


In [2]:
# Init cluster (describes a single node, or less if need less than that per worker)
# cluster = SLURMCluster(
#     cores=56,
#     processes=4,  # check docs
#     memory="192GB",
#     account="UKAEA-AP002-CPU",
#     walltime="01:00:00",
#     queue="cclake",
# )

# Need less than a full node per worker
cluster = SLURMCluster(
    cores=1,
    processes=1,
    memory="4GB",
    account="UKAEA-AP002-CPU",
    walltime="20:00:00",
    queue="cclake",
)
# cluster.scale(jobs=4)
cluster.adapt(minimum_jobs=10,maximum_jobs=300)  # 4 workers
# print(cluster.job_script())

# Connect Dask client to remote cluster
client = Client(cluster)
# Code from now on submitted to batch queue

# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_cluster", work_dir=WORK_DIR)


# Define parameter space
# The ranges here are taken from demo_confopt_1, the first optimisation in the previous study.
params = {
    "coreradius": {
        "type": "float",
        "min": 0.6,
        "max": 0.9,
        "default": 0.7
    },
    "ralpne": {
        "type": "float",
        "min": 0.072,
        "max": 0.0761,
        "default": 0.74
    }, 
    "psepbqarmax": {
        "type": "float",
        "min": 5.0,
        "max": 14.04,
        "default": 9.0
    },
    "tbrnmn": {
        "type": "float",
        "min": 6000.0,
        "max": 60000.0,
        "default": 10000.0 
    },
    "etaech": {
        "type": "float",
        "min": 0.20,
        "max": 0.55,
        "default":0.40
    }, 
    "pinjalw": {
        "type": "float",
        "min": 40.0,
        "max": 60.0,
        "default": 50.0
    }, 
    "triang": {
        "type": "float",
        "min": 0.4,
        "max": 0.6,
        "default":0.5
    }, 
    "alstroh": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default":628000000.0 
    }, 
    "sig_tf_case_max": {
        "type": "float",
        "min": 464000000.,
        "max": 696000000.0,
        "default": 596000000.0
    }, 
    "walalw": {
        "type": "float",
        "min": 4.0,
        "max": 12.0,
        "default": 7.4
    }, 
    "sig_tf_wp_max": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default": 692000000.0
    }, 
    "aspect": {
        "type": "float",
        "min": 2.4,
        "max": 3.5,
        "default": 2.82
    }, 
    "etath": {
        "type": "float",
        "min": 0.3,
        "max": 0.5,
        "default": 0.375
    }, 
    "n_cycle_min": {
        "type": "float",
        "min": 20000.0,
        "max": 40000.0,
        "default": 20000.00
    },
    "vary_param": {
        "type": "string",
        "default": "",
    },  
}

# QoIs
# A list of iteration variables, ifail and sqsumsq.
qois = [
        "ifail",
        "sqsumsq",
        "bt",
        "te",
        "beta",
        "dene",
        "tfcth",
        "wallmw",
        "ohcth",
        "bigq",
        "bore",
        "betalim",
        "coheof",
        "cohbop",
        "kappa",
        "fvsbrnni",
        "itvar019",
        "itvar020",
        "jwptf",
        "vtfskv",
        "vdalw",
        "tdmptf",
        "thkcas",
        "thwcndut",
        "fcutfsu",
        "cpttf",
        "plhthresh",
        "tmargtf",
        "tmargoh",
        "oh_steel_frac",
        "pdivt",
        "powfmw",
        "rmajor",
        "n_cycle"
]

# Create encoder and decoder
encoder1 = uq.encoders.GenericEncoder(
    template_fname="clean_demo_2023_IN.template", target_filename="IN.DAT"
)
encoder2 = uq.encoders.GenericEncoder(
    template_fname="MFILE.DAT.json", target_filename="MFILE.DAT.json"
)
decoder = uq.decoders.JSONDecoder(target_filename="MFILE.DAT.json", output_columns=qois)
my_multiencoder = uq.encoders.MultiEncoder(encoder1, encoder2)


cmd = "process -i IN.DAT --mfilejson --mfile MFILE.DAT"
actions = uq.actions.local_execute(my_multiencoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

# Create PCE sampler, 4 uncertainties
# vary = {
#     "aspect": cp.Uniform(2.785,2.935),
#     "triang": cp.Uniform(0.4, 0.6),
#     "psepbqarmax": cp.Uniform(8.7, 9.7),
#     "hfact": cp.Uniform(1.0, 1.2),
# }
vary={"coreradius": cp.Uniform(0.6,0.9),
      "ralpne": cp.Uniform(0.072,0.0761),
      "psepbqarmax": cp.Uniform(7.0,11.02),
      "tbrnmn": cp.Uniform(12800.0,15700.0),
      "etaech": cp.Uniform(0.26,0.51),
      "pinjalw": cp.Uniform(40.0,60.0),
      "triang": cp.Uniform(0.4,0.5),
      "alstroh": cp.Uniform(600000000.0, 720000000.0),
      "sig_tf_case_max": cp.Uniform(520000000.0, 640000000.0),
      "walalw": cp.Uniform(5.0, 10.0),
      "sig_tf_wp_max": cp.Uniform(528000000.0, 792000000.0),
      "aspect": cp.Uniform(2.75, 2.90),
      "etath": cp.Uniform(0.40, 0.4125),
      "n_cycle_min": cp.Uniform(20000.00,40000.00)
      }
mc_sampler = uq.sampling.MCSampler(vary=vary,n_mc_samples=250)

# Add mc_sampler to campaign
campaign.set_sampler(mc_sampler)

# Draw samples, execute and collate
campaign.execute(pool=client).collate(progress_bar=True)
samples = campaign.get_collation_result()
samples

# Specify the file path where you want to save the HDF file
hdf_file_path = 'uncertainties_data.h5'

# Save the DataFrame to HDF
samples.to_hdf(hdf_file_path, key='data', mode='w')



  0%|          | 0/700 [00:00<?, ?it/s]


 11%|█         | 76/700 [00:00<00:00, 759.90it/s]


 38%|███▊      | 263/700 [00:00<00:00, 1410.21it/s]


 63%|██████▎   | 442/700 [00:00<00:00, 1580.81it/s]


 88%|████████▊ | 613/700 [00:00<00:00, 1629.43it/s]


100%|██████████| 700/700 [00:00<00:00, 1535.30it/s]